In [33]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

memory.save_context({"input": "hi"}, {"output": "How are you?"})

memory.load_memory_variables({})

{'history': 'Human: hi\nAI: How are you?'}

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

chat = ChatOpenAI(
    temperature=0.1,
    # streaming=True, callbacks=[StreamingStdOutCallbackHandler()]
)

with get_openai_callback() as usage:
    a = chat.predict("소주는 어떻게 만드니?")
    b = chat.predict("빵은 어떻게 만드니?")
    print(a, "\n", b, "\n")
    print(usage)

소주는 보리나 감자, 쌀 등의 원료를 발효시켜 만들어지는 전통적인 한국 주류입니다. 먼저 선택한 원료를 발효시켜 알코올을 얻은 후, 증류과정을 거쳐 순수한 주정을 얻습니다. 이후 숙성을 거쳐 부드럽고 깔끔한 맛을 내는 과정을 거쳐 완성됩니다. 소주의 맛과 향은 발효 및 증류과정에서 사용되는 효모와 곡물의 종류, 숙성기간 등에 따라 다양하게 변화합니다. 
 빵을 만드는 과정은 다음과 같습니다:

1. 재료 준비: 밀가루, 물, 이스트, 설탕, 소금, 버터 등의 재료를 준비합니다.

2. 반죽: 밀가루, 이스트, 물, 설탕, 소금을 섞어 반죽을 만듭니다. 이때, 반죽이 부드럽고 탄력이 있도록 충분히 섞어야 합니다.

3. 발효: 반죽을 따뜻한 곳에 두어 이스트가 발효되도록 합니다. 이스트가 발효되면 반죽이 부풀어 오르게 됩니다.

4. 성형: 발효된 반죽을 모양에 맞게 성형합니다. 둥근 모양의 빵, 잎 모양의 빵 등 다양한 모양으로 만들 수 있습니다.

5. 굽기: 성형된 반죽을 오븐에 넣어 굽습니다. 굽는 동안 반죽이 부풀어 오르고 골드 브라운 컬러가 나타날 때까지 굽습니다.

6. 식히기: 굽은 빵을 꺼내어 식혀서 완전히 식힙니다.

7. 마무리: 식힌 빵에 버터나 잼을 발라서 맛을 낸 후, 준비가 완료됩니다.

이렇게 만들어진 빵은 다양한 종류의 빵으로 변신할 수 있습니다. 예를 들어, 크로와상, 베이글, 호밀빵, 밀크토스트 등 다양한 빵을 만들 수 있습니다. 

Tokens Used: 747
	Prompt Tokens: 41
	Completion Tokens: 706
Successful Requests: 2
Total Cost (USD): $0.0014735


In [22]:
from langchain.llms.openai import OpenAI
from langchain.llms.loading import load_llm

# chat = OpenAI(temperature=0.1, max_tokens=450, model="gpt-4o-mini")

# chat.save("model.json")

chat = load_llm("model.json")

chat

c:\Users\woosu\Dev\python\fullstack-gpt\env\Lib\site-packages\langchain\llms\openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
c:\Users\woosu\Dev\python\fullstack-gpt\env\Lib\site-packages\langchain\llms\openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


OpenAIChat(client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-4o-mini', model_kwargs={'temperature': 0.1, 'max_tokens': 450, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'request_timeout': None, 'logit_bias': {}})